In [ ]:
import numpy as np
import xarray as xr
import os,errno
import sys
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import salem
import scipy.stats as st

dir_data='/Users/ahenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

In [ ]:
yrs=np.arange(1980,2024,1)

ds=xr.open_dataset(dir_data+'merra2_gridareas.nc')
areas=ds['cell_area']/1e6

nums_array_merra2_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
areas_array_merra2_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
lats_array_merra2_polar=[[],[],[],[],[],[],[],[],[],[],[],[]]
nums_array_merra2_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
areas_array_merra2_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
lats_array_merra2_fixed=[[],[],[],[],[],[],[],[],[],[],[],[]]
nums_array_merra2_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
areas_array_merra2_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]
lats_array_merra2_fixed_poleward=[[],[],[],[],[],[],[],[],[],[],[],[]]

lats_array_merra2_polar_nh=[[],[],[],[],[],[],[],[],[],[],[],[]]
lats_array_merra2_fixed_nh=[[],[],[],[],[],[],[],[],[],[],[],[]]
lats_array_merra2_fixed_poleward_nh=[[],[],[],[],[],[],[],[],[],[],[],[]]
lats_array_merra2_polar_sh=[[],[],[],[],[],[],[],[],[],[],[],[]]
lats_array_merra2_fixed_sh=[[],[],[],[],[],[],[],[],[],[],[],[]]
lats_array_merra2_fixed_poleward_sh=[[],[],[],[],[],[],[],[],[],[],[],[]]

nums_list_merra2_polar=[]
areas_list_merra2_polar=[]
lats_list_merra2_polar=[]
nums_list_merra2_fixed=[]
areas_list_merra2_fixed=[]
lats_list_merra2_fixed=[]

nums_list_merra2_fixed_poleward=[]
areas_list_merra2_fixed_poleward=[]
lats_list_merra2_fixed_poleward=[]
lats_list_merra2_polar_nh=[]
lats_list_merra2_fixed_nh=[]
lats_list_merra2_fixed_poleward_nh=[]
lats_list_merra2_polar_sh=[]
lats_list_merra2_fixed_sh=[]
lats_list_merra2_fixed_poleward_sh=[]


for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    if year>=1980:
        ds=xr.open_dataset(dir2+'merra2.ar.labels.model.variable.'+str(year)+'.nc')
        labeled=ds['ar_labeled']

        if i==0:
            lat_array=xr.zeros_like(labeled[0,:,:])
            lon_array=xr.zeros_like(labeled[0,:,:])
            for k in range(labeled.lon.size):
                lat_array[:,k]=labeled.lat.values
            for k in range(labeled.lat.size):
                lon_array[k,:]=labeled.lon.values

            weights=np.cos(np.deg2rad(ds.lat))

        num_list_year=[]
        areas_list_year=[]
        lats_list_year=[]
        lats_list_year_nh=[]
        lats_list_year_sh=[]

        nums_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        areas_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        lats_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        lats_array_year_nh=[[],[],[],[],[],[],[],[],[],[],[],[]]
        lats_array_year_sh=[[],[],[],[],[],[],[],[],[],[],[],[]]

        count=0
        for j in range(labeled.time.size):
            date=pd.to_datetime(labeled.time.values[j])
            month=date.month

            label_sel=labeled[j,:,:]
            unique=np.unique(label_sel.values)
            unique=[x for x in unique if x>0]
            num=len(unique)

            num_day=0
            for k in range(len(unique)):
                sel_feature=label_sel.where(label_sel==unique[k])
                area_feature=areas.where(sel_feature>0)
                area_feature=area_feature.sum(skipna=True).values.tolist()
                lats_sel=lat_array.where(label_sel==unique[k])
                lats_weighted=lats_sel.weighted(weights)
                lat_mean=lats_weighted.mean(skipna=True).values.tolist()

                #print(area_feature)
                #if area_feature>100000:
                if area_feature>0:
                    areas_list_year.append(area_feature)
                    lats_list_year.append(abs(lat_mean))
                    areas_array_year[month-1].append(area_feature)
                    lats_array_year[month-1].append(abs(lat_mean))
                    
                    if lat_mean>0:
                        lats_list_year_nh.append(lat_mean)
                        lats_array_year_nh[month-1].append(lat_mean)
                        
                    if lat_mean<0:
                        lats_list_year_sh.append(lat_mean)
                        lats_array_year_sh[month-1].append(lat_mean)
                        
                    num_day=num_day+1

            num_list_year.append(num_day)
            nums_array_year[month-1].append(num_day)

        mean_num=float(sum(num_list_year))/float(len(num_list_year))
        mean_area=float(sum(areas_list_year))/float(len(areas_list_year))
        mean_lat=float(sum(lats_list_year))/float(len(lats_list_year))
        nums_list_merra2_polar.append(mean_num)
        areas_list_merra2_polar.append(mean_area)
        lats_list_merra2_polar.append(mean_lat)
        
        mean_lat_nh=float(sum(lats_list_year_nh))/float(len(lats_list_year_nh))
        mean_lat_sh=float(sum(lats_list_year_sh))/float(len(lats_list_year_sh))
        lats_list_merra2_polar_nh.append(mean_lat_nh)
        lats_list_merra2_polar_sh.append(mean_lat_sh)

        for j in range(len(nums_array_year)):
            nums_array_merra2_polar[j].append(float(sum(nums_array_year[j]))/float(len(nums_array_year[j])))#mean of all time step AR numbers
            areas_array_merra2_polar[j].append(float(sum(areas_array_year[j]))/float(len(areas_array_year[j])))#mean of all areas in month
            lats_array_merra2_polar[j].append(float(sum(lats_array_year[j]))/float(len(lats_array_year[j])))#mean of all avg lats in month
            lats_array_merra2_polar_nh[j].append(float(sum(lats_array_year_nh[j]))/float(len(lats_array_year_nh[j])))#mean of all avg lats in month
            lats_array_merra2_polar_sh[j].append(float(sum(lats_array_year_sh[j]))/float(len(lats_array_year_sh[j])))#mean of all avg lats in month

        #################

        ds=xr.open_dataset(dir2+'merra2.ar.labels.model.'+str(year)+'.nc')
        labeled=ds['ar_labeled'][0::2,:,:]

        if i==0:
            lat_array=xr.zeros_like(labeled[0,:,:])
            lon_array=xr.zeros_like(labeled[0,:,:])
            for k in range(labeled.lon.size):
                lat_array[:,k]=labeled.lat.values
            for k in range(labeled.lat.size):
                lon_array[k,:]=labeled.lon.values

            weights=np.cos(np.deg2rad(ds.lat))

        num_list_year=[]
        areas_list_year=[]
        lats_list_year=[]
        lats_list_year_nh=[]
        lats_list_year_sh=[]

        nums_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        areas_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        lats_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        lats_array_year_nh=[[],[],[],[],[],[],[],[],[],[],[],[]]
        lats_array_year_sh=[[],[],[],[],[],[],[],[],[],[],[],[]]

        count=0
        for j in range(labeled.time.size):
            date=pd.to_datetime(labeled.time.values[j])
            month=date.month

            label_sel=labeled[j,:,:]
            unique=np.unique(label_sel.values)
            unique=[x for x in unique if x>0]
            num=len(unique)

            num_day=0
            for k in range(len(unique)):
                sel_feature=label_sel.where(label_sel==unique[k])
                area_feature=areas.where(sel_feature>0)
                area_feature=area_feature.sum(skipna=True).values.tolist()
                lats_sel=lat_array.where(label_sel==unique[k])
                lats_weighted=lats_sel.weighted(weights)
                lat_mean=lats_weighted.mean(skipna=True).values.tolist()

                #if area_feature>100000:
                if area_feature>0:
                    areas_list_year.append(area_feature)
                    lats_list_year.append(abs(lat_mean))
                    areas_array_year[month-1].append(area_feature)
                    lats_array_year[month-1].append(abs(lat_mean))
                    
                    if lat_mean>0:
                        lats_list_year_nh.append(lat_mean)
                        lats_array_year_nh[month-1].append(lat_mean)
                        
                    if lat_mean<0:
                        lats_list_year_sh.append(lat_mean)
                        lats_array_year_sh[month-1].append(lat_mean)
                        
                    num_day=num_day+1

            num_list_year.append(num_day)
            nums_array_year[month-1].append(num_day)

        mean_num=float(sum(num_list_year))/float(len(num_list_year))
        mean_area=float(sum(areas_list_year))/float(len(areas_list_year))
        mean_lat=float(sum(lats_list_year))/float(len(lats_list_year))
        nums_list_merra2_fixed.append(mean_num)
        areas_list_merra2_fixed.append(mean_area)
        lats_list_merra2_fixed.append(mean_lat)
        
        mean_lat_nh=float(sum(lats_list_year_nh))/float(len(lats_list_year_nh))
        mean_lat_sh=float(sum(lats_list_year_sh))/float(len(lats_list_year_sh))
        lats_list_merra2_fixed_nh.append(mean_lat_nh)
        lats_list_merra2_fixed_sh.append(mean_lat_sh)

        for j in range(len(nums_array_year)):
            nums_array_merra2_fixed[j].append(float(sum(nums_array_year[j]))/float(len(nums_array_year[j])))#mean of all time step AR numbers
            areas_array_merra2_fixed[j].append(float(sum(areas_array_year[j]))/float(len(areas_array_year[j])))#mean of all areas in month
            lats_array_merra2_fixed[j].append(float(sum(lats_array_year[j]))/float(len(lats_array_year[j])))#mean of all avg lats in month
            lats_array_merra2_fixed_nh[j].append(float(sum(lats_array_year_nh[j]))/float(len(lats_array_year_nh[j])))#mean of all avg lats in month
            lats_array_merra2_fixed_sh[j].append(float(sum(lats_array_year_sh[j]))/float(len(lats_array_year_sh[j])))#mean of all avg lats in month

            
        #####
        
        ds=xr.open_dataset(dir2+'merra2.ar.labels.model.poleward.'+str(year)+'.nc')
        labeled=ds['ar_labeled']

        if i==0:
            lat_array=xr.zeros_like(labeled[0,:,:])
            lon_array=xr.zeros_like(labeled[0,:,:])
            for k in range(labeled.lon.size):
                lat_array[:,k]=labeled.lat.values
            for k in range(labeled.lat.size):
                lon_array[k,:]=labeled.lon.values

            weights=np.cos(np.deg2rad(ds.lat))

        num_list_year=[]
        areas_list_year=[]
        lats_list_year=[]
        lats_list_year_nh=[]
        lats_list_year_sh=[]

        nums_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        areas_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        lats_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        lats_array_year_nh=[[],[],[],[],[],[],[],[],[],[],[],[]]
        lats_array_year_sh=[[],[],[],[],[],[],[],[],[],[],[],[]]

        count=0
        for j in range(labeled.time.size):
            date=pd.to_datetime(labeled.time.values[j])
            month=date.month

            label_sel=labeled[j,:,:]
            unique=np.unique(label_sel.values)
            unique=[x for x in unique if x>0]
            num=len(unique)

            num_day=0
            for k in range(len(unique)):
                sel_feature=label_sel.where(label_sel==unique[k])
                area_feature=areas.where(sel_feature>0)
                area_feature=area_feature.sum(skipna=True).values.tolist()
                lats_sel=lat_array.where(label_sel==unique[k])
                lats_weighted=lats_sel.weighted(weights)
                lat_mean=lats_weighted.mean(skipna=True).values.tolist()

                #if area_feature>100000:
                if area_feature>0:
                    areas_list_year.append(area_feature)
                    lats_list_year.append(abs(lat_mean))
                    areas_array_year[month-1].append(area_feature)
                    lats_array_year[month-1].append(abs(lat_mean))
                    
                    if lat_mean>0:
                        lats_list_year_nh.append(lat_mean)
                        lats_array_year_nh[month-1].append(lat_mean)
                        
                    if lat_mean<0:
                        lats_list_year_sh.append(lat_mean)
                        lats_array_year_sh[month-1].append(lat_mean)
                        
                    num_day=num_day+1

            num_list_year.append(num_day)
            nums_array_year[month-1].append(num_day)

        mean_num=float(sum(num_list_year))/float(len(num_list_year))
        mean_area=float(sum(areas_list_year))/float(len(areas_list_year))
        mean_lat=float(sum(lats_list_year))/float(len(lats_list_year))
        nums_list_merra2_fixed_poleward.append(mean_num)
        areas_list_merra2_fixed_poleward.append(mean_area)
        lats_list_merra2_fixed_poleward.append(mean_lat)
        
        mean_lat_nh=float(sum(lats_list_year_nh))/float(len(lats_list_year_nh))
        mean_lat_sh=float(sum(lats_list_year_sh))/float(len(lats_list_year_sh))
        lats_list_merra2_fixed_poleward_nh.append(mean_lat_nh)
        lats_list_merra2_fixed_poleward_sh.append(mean_lat_sh)

        for j in range(len(nums_array_year)):
            nums_array_merra2_fixed_poleward[j].append(float(sum(nums_array_year[j]))/float(len(nums_array_year[j])))#mean of all time step AR numbers
            areas_array_merra2_fixed_poleward[j].append(float(sum(areas_array_year[j]))/float(len(areas_array_year[j])))#mean of all areas in month
            lats_array_merra2_fixed_poleward[j].append(float(sum(lats_array_year[j]))/float(len(lats_array_year[j])))#mean of all avg lats in month
            lats_array_merra2_fixed_poleward_nh[j].append(float(sum(lats_array_year_nh[j]))/float(len(lats_array_year_nh[j])))#mean of all avg lats in month
            lats_array_merra2_fixed_poleward_sh[j].append(float(sum(lats_array_year_sh[j]))/float(len(lats_array_year_sh[j])))#mean of all avg lats in month

In [ ]:
dk=xr.Dataset()

dk['nums_array_merra2_polar']=(('months','years'),np.asarray(nums_array_merra2_polar))
dk['areas_array_merra2_polar']=(('months','years'),np.asarray(areas_array_merra2_polar))
dk['lats_array_merra2_polar']=(('months','years'),np.asarray(lats_array_merra2_polar))
dk['lats_array_merra2_polar_nh']=(('months','years'),np.asarray(lats_array_merra2_polar_nh))
dk['lats_array_merra2_polar_sh']=(('months','years'),np.asarray(lats_array_merra2_polar_sh))

dk['nums_array_merra2_fixed']=(('months','years'),np.asarray(nums_array_merra2_fixed))
dk['areas_array_merra2_fixed']=(('months','years'),np.asarray(areas_array_merra2_fixed))
dk['lats_array_merra2_fixed']=(('months','years'),np.asarray(lats_array_merra2_fixed))
dk['lats_array_merra2_fixed_nh']=(('months','years'),np.asarray(lats_array_merra2_fixed_nh))
dk['lats_array_merra2_fixed_sh']=(('months','years'),np.asarray(lats_array_merra2_fixed_sh))

dk['nums_array_merra2_fixed_poleward']=(('months','years'),np.asarray(nums_array_merra2_fixed_poleward))
dk['areas_array_merra2_fixed_poleward']=(('months','years'),np.asarray(areas_array_merra2_fixed_poleward))
dk['lats_array_merra2_fixed_poleward']=(('months','years'),np.asarray(lats_array_merra2_fixed_poleward))
dk['lats_array_merra2_fixed_poleward_nh']=(('months','years'),np.asarray(lats_array_merra2_fixed_poleward_nh))
dk['lats_array_merra2_fixed_poleward_sh']=(('months','years'),np.asarray(lats_array_merra2_fixed_poleward_sh))

dk['nums_list_merra2_polar']=(('years'),np.asarray(nums_list_merra2_polar))
dk['areas_list_merra2_polar']=(('years'),np.asarray(areas_list_merra2_polar))
dk['lats_list_merra2_polar']=(('years'),np.asarray(lats_list_merra2_polar))
dk['lats_list_merra2_polar_nh']=(('years'),np.asarray(lats_list_merra2_polar_nh))
dk['lats_list_merra2_polar_sh']=(('years'),np.asarray(lats_list_merra2_polar_sh))

dk['nums_list_merra2_fixed']=(('years'),np.asarray(nums_list_merra2_fixed))
dk['areas_list_merra2_fixed']=(('years'),np.asarray(areas_list_merra2_fixed))
dk['lats_list_merra2_fixed']=(('years'),np.asarray(lats_list_merra2_fixed))
dk['lats_list_merra2_fixed_nh']=(('years'),np.asarray(lats_list_merra2_fixed_nh))
dk['lats_list_merra2_fixed_sh']=(('years'),np.asarray(lats_list_merra2_fixed_sh))

dk['nums_list_merra2_fixed_poleward']=(('years'),np.asarray(nums_list_merra2_fixed_poleward))
dk['areas_list_merra2_fixed_poleward']=(('years'),np.asarray(areas_list_merra2_fixed_poleward))
dk['lats_list_merra2_fixed_poleward']=(('years'),np.asarray(lats_list_merra2_fixed_poleward))
dk['lats_list_merra2_fixed_poleward_nh']=(('years'),np.asarray(lats_list_merra2_fixed_poleward_nh))
dk['lats_list_merra2_fixed_poleward_sh']=(('years'),np.asarray(lats_list_merra2_fixed_poleward_sh))

dk.coords['years']=np.arange(1980,2024,1)
dk.coords['months']=np.arange(1,13,1)
try:
    os.remove(dir_data+'ar_chars_original_merra2.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'ar_chars_original_merra2.nc',mode='w',format='NETCDF4')

In [ ]:
ds=xr.open_dataset(dir_data+'ar_chars_original_merra2.nc')#era5, merra2 

nums_list_merra2_polar=ds['nums_list_merra2_polar'].values.tolist()
areas_list_merra2_polar=ds['areas_list_merra2_polar'].values.tolist()
lats_list_merra2_polar=ds['lats_list_merra2_polar'].values.tolist()
lats_list_merra2_polar_nh=ds['lats_list_merra2_polar_nh'].values.tolist()
lats_list_merra2_polar_sh=[x*-1. for x in ds['lats_list_merra2_polar_sh'].values.tolist()]

nums_list_merra2_fixed=ds['nums_list_merra2_fixed'].values.tolist()
areas_list_merra2_fixed=ds['areas_list_merra2_fixed'].values.tolist()
lats_list_merra2_fixed=ds['lats_list_merra2_fixed'].values.tolist()
lats_list_merra2_fixed_nh=ds['lats_list_merra2_fixed_nh'].values.tolist()
lats_list_merra2_fixed_sh=[x*-1. for x in ds['lats_list_merra2_fixed_sh'].values.tolist()]

nums_list_merra2_fixed_poleward=ds['nums_list_merra2_fixed_poleward'].values.tolist()
areas_list_merra2_fixed_poleward=ds['areas_list_merra2_fixed_poleward'].values.tolist()
lats_list_merra2_fixed_poleward=ds['lats_list_merra2_fixed_poleward'].values.tolist()
lats_list_merra2_fixed_poleward_nh=ds['lats_list_merra2_fixed_poleward_nh'].values.tolist()
lats_list_merra2_fixed_poleward_sh=[x*-1. for x in ds['lats_list_merra2_fixed_poleward_sh'].values.tolist()]

nums_array_merra2_polar=ds['nums_array_merra2_polar'].values
areas_array_merra2_polar=ds['areas_array_merra2_polar'].values
lats_array_merra2_polar=ds['lats_array_merra2_polar'].values
lats_array_merra2_polar_nh=ds['lats_array_merra2_polar_nh'].values
lats_array_merra2_polar_sh=ds['lats_array_merra2_polar_sh'].values*-1.

nums_array_merra2_fixed=ds['nums_array_merra2_fixed'].values
areas_array_merra2_fixed=ds['areas_array_merra2_fixed'].values
lats_array_merra2_fixed=ds['lats_array_merra2_fixed'].values
lats_array_merra2_fixed_nh=ds['lats_array_merra2_fixed_nh'].values
lats_array_merra2_fixed_sh=ds['lats_array_merra2_fixed_sh'].values*-1.

nums_array_merra2_fixed_poleward=ds['nums_array_merra2_fixed_poleward'].values
areas_array_merra2_fixed_poleward=ds['areas_array_merra2_fixed_poleward'].values
lats_array_merra2_fixed_poleward=ds['lats_array_merra2_fixed_poleward'].values
lats_array_merra2_fixed_poleward_nh=ds['lats_array_merra2_fixed_poleward_nh'].values
lats_array_merra2_fixed_poleward_sh=ds['lats_array_merra2_fixed_poleward_sh'].values*-1.

ds=xr.open_dataset(dir_data+'ar_chars_original_era5.nc')#era5, merra2 

nums_list_era5_polar=ds['nums_list_era5_polar'].values.tolist()
areas_list_era5_polar=ds['areas_list_era5_polar'].values.tolist()
lats_list_era5_polar=ds['lats_list_era5_polar'].values.tolist()
lats_list_era5_polar_nh=ds['lats_list_era5_polar_nh'].values.tolist()
lats_list_era5_polar_sh=[x*-1. for x in ds['lats_list_era5_polar_sh'].values.tolist()]

nums_list_era5_fixed=ds['nums_list_era5_fixed'].values.tolist()
areas_list_era5_fixed=ds['areas_list_era5_fixed'].values.tolist()
lats_list_era5_fixed=ds['lats_list_era5_fixed'].values.tolist()
lats_list_era5_fixed_nh=ds['lats_list_era5_fixed_nh'].values.tolist()
lats_list_era5_fixed_sh=[x*-1. for x in ds['lats_list_era5_fixed_sh'].values.tolist()]

nums_list_era5_fixed_poleward=ds['nums_list_era5_fixed_poleward'].values.tolist()
areas_list_era5_fixed_poleward=ds['areas_list_era5_fixed_poleward'].values.tolist()
lats_list_era5_fixed_poleward=ds['lats_list_era5_fixed_poleward'].values.tolist()
lats_list_era5_fixed_poleward_nh=ds['lats_list_era5_fixed_poleward_nh'].values.tolist()
lats_list_era5_fixed_poleward_sh=[x*-1. for x in ds['lats_list_era5_fixed_poleward_sh'].values.tolist()]

nums_array_era5_polar=ds['nums_array_era5_polar'].values
areas_array_era5_polar=ds['areas_array_era5_polar'].values
lats_array_era5_polar=ds['lats_array_era5_polar'].values
lats_array_era5_polar_nh=ds['lats_array_era5_polar_nh'].values
lats_array_era5_polar_sh=ds['lats_array_era5_polar_sh'].values*-1.

nums_array_era5_fixed=ds['nums_array_era5_fixed'].values
areas_array_era5_fixed=ds['areas_array_era5_fixed'].values
lats_array_era5_fixed=ds['lats_array_era5_fixed'].values
lats_array_era5_fixed_nh=ds['lats_array_era5_fixed_nh'].values
lats_array_era5_fixed_sh=ds['lats_array_era5_fixed_sh'].values*-1.

nums_array_era5_fixed_poleward=ds['nums_array_era5_fixed_poleward'].values
areas_array_era5_fixed_poleward=ds['areas_array_era5_fixed_poleward'].values
lats_array_era5_fixed_poleward=ds['lats_array_era5_fixed_poleward'].values
lats_array_era5_fixed_poleward_nh=ds['lats_array_era5_fixed_poleward_nh'].values
lats_array_era5_fixed_poleward_sh=ds['lats_array_era5_fixed_poleward_sh'].values

ds=xr.open_dataset(dir_data+'ar_chars_original_jra55.nc')#era5, merra2 

nums_list_jra55_polar=ds['nums_list_jra55_polar'].values.tolist()
areas_list_jra55_polar=ds['areas_list_jra55_polar'].values.tolist()
lats_list_jra55_polar=ds['lats_list_jra55_polar'].values.tolist()
lats_list_jra55_polar_nh=ds['lats_list_jra55_polar_nh'].values.tolist()
lats_list_jra55_polar_sh=[x*-1. for x in ds['lats_list_jra55_polar_sh'].values.tolist()]

nums_list_jra55_fixed=ds['nums_list_jra55_fixed'].values.tolist()
areas_list_jra55_fixed=ds['areas_list_jra55_fixed'].values.tolist()
lats_list_jra55_fixed=ds['lats_list_jra55_fixed'].values.tolist()
lats_list_jra55_fixed_nh=ds['lats_list_jra55_fixed_nh'].values.tolist()
lats_list_jra55_fixed_sh=[x*-1. for x in ds['lats_list_jra55_fixed_sh'].values.tolist()]

nums_list_jra55_fixed_poleward=ds['nums_list_jra55_fixed_poleward'].values.tolist()
areas_list_jra55_fixed_poleward=ds['areas_list_jra55_fixed_poleward'].values.tolist()
lats_list_jra55_fixed_poleward=ds['lats_list_jra55_fixed_poleward'].values.tolist()
lats_list_jra55_fixed_poleward_nh=ds['lats_list_jra55_fixed_poleward_nh'].values.tolist()
lats_list_jra55_fixed_poleward_sh=[x*-1. for x in ds['lats_list_jra55_fixed_poleward_sh'].values.tolist()]

nums_array_jra55_polar=ds['nums_array_jra55_polar'].values
areas_array_jra55_polar=ds['areas_array_jra55_polar'].values
lats_array_jra55_polar=ds['lats_array_jra55_polar'].values
lats_array_jra55_polar_nh=ds['lats_array_jra55_polar_nh'].values
lats_array_jra55_polar_sh=ds['lats_array_jra55_polar_sh'].values*-1.

nums_array_jra55_fixed=ds['nums_array_jra55_fixed'].values
areas_array_jra55_fixed=ds['areas_array_jra55_fixed'].values
lats_array_jra55_fixed=ds['lats_array_jra55_fixed'].values
lats_array_jra55_fixed_nh=ds['lats_array_jra55_fixed_nh'].values
lats_array_jra55_fixed_sh=ds['lats_array_jra55_fixed_sh'].values*-1.

nums_array_jra55_fixed_poleward=ds['nums_array_jra55_fixed_poleward'].values
areas_array_jra55_fixed_poleward=ds['areas_array_jra55_fixed_poleward'].values
lats_array_jra55_fixed_poleward=ds['lats_array_jra55_fixed_poleward'].values
lats_array_jra55_fixed_poleward_nh=ds['lats_array_jra55_fixed_poleward_nh'].values
lats_array_jra55_fixed_poleward_sh=ds['lats_array_jra55_fixed_poleward_sh'].values*-1.

In [ ]:
yrs=np.arange(1940,2024,1)
for i in range(len(yrs)):
    if i==0:
        nums_month_era5_polar=list(np.asarray(nums_array_era5_polar)[:,i])
        nums_month_era5_fixed=list(np.asarray(nums_array_era5_fixed)[:,i])
        nums_month_era5_fixed_poleward=list(np.asarray(nums_array_era5_fixed_poleward)[:,i])
        
    else:
        nums_month_era5_polar=nums_month_era5_polar+list(np.asarray(nums_array_era5_polar)[:,i])
        nums_month_era5_fixed=nums_month_era5_fixed+list(np.asarray(nums_array_era5_fixed)[:,i])
        nums_month_era5_fixed_poleward=nums_month_era5_fixed_poleward+list(np.asarray(nums_array_era5_fixed_poleward)[:,i])

yrs=np.arange(1980,2024,1)
for i in range(len(yrs)):
    if i==0:
        nums_month_merra2_polar=list(np.asarray(nums_array_merra2_polar)[:,i-1])
        nums_month_merra2_fixed=list(np.asarray(nums_array_merra2_fixed)[:,i-1])
        nums_month_merra2_fixed_poleward=list(np.asarray(nums_array_merra2_fixed_poleward)[:,i-1])
        
    else:
        nums_month_merra2_polar=nums_month_merra2_polar+list(np.asarray(nums_array_merra2_polar)[:,i-1])
        nums_month_merra2_fixed=nums_month_merra2_fixed+list(np.asarray(nums_array_merra2_fixed)[:,i-1])
        nums_month_merra2_fixed_poleward=nums_month_merra2_fixed_poleward+list(np.asarray(nums_array_merra2_fixed_poleward)[:,i-1])

yrs=np.arange(1958,2024,1)
for i in range(len(yrs)):
    if i==0:
        nums_month_jra55_polar=list(np.asarray(nums_array_jra55_polar)[:,i-1])
        nums_month_jra55_fixed=list(np.asarray(nums_array_jra55_fixed)[:,i-1])
        nums_month_jra55_fixed_poleward=list(np.asarray(nums_array_jra55_fixed_poleward)[:,i-1])
        
    else:
        nums_month_jra55_polar=nums_month_jra55_polar+list(np.asarray(nums_array_jra55_polar)[:,i-1])
        nums_month_jra55_fixed=nums_month_jra55_fixed+list(np.asarray(nums_array_jra55_fixed)[:,i-1])
        nums_month_jra55_fixed_poleward=nums_month_jra55_fixed_poleward+list(np.asarray(nums_array_jra55_fixed_poleward)[:,i-1])

        
x_arr=np.arange(1979.5,2023.5,1./12.)


month_days=[31,28.25,31,30,31,30,31,31,30,31,30,31]

sigs_list=[]

yrs=np.arange(1980,2024,1)
test=[x/1. for x in nums_list_merra2_fixed]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in nums_list_merra2_polar]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='MERRA-2 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in nums_list_merra2_fixed_poleward]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='MERRA-2 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs1=np.arange(1940+40,2024,1)
test=[x/1. for x in nums_list_era5_fixed[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_4='ERA5 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in nums_list_era5_polar[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_5='ERA5 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in nums_list_era5_fixed_poleward[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_6='ERA5 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs2=np.arange(1958+22,2024,1)
test=[x/1. for x in nums_list_jra55_fixed[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_7='JRA-55 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in nums_list_jra55_polar[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_8='JRA-55 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in nums_list_jra55_fixed_poleward[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_9='JRA-55 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'


yrs1=np.arange(1940,2024,1)
yrs2=np.arange(1958,2024,1)

fig=plt.figure(figsize=(9,15))
ax=plt.subplot(1,1,1)

#MERRA-2: polar=salmon, fixed = yellow, fixed poleward = deepskyblue
#JRA-55: polar = r, fixed = gold, fixed poleward = royalblue
#ERA5: polar = darkred, fixed = orange, fixed poleward = mediumblue

width=0.75
ax.plot(x_arr,nums_month_merra2_polar,color='salmon',linewidth=width,alpha=0.5)
ax.plot(x_arr,nums_month_merra2_fixed,color='yellow',linewidth=width,alpha=0.5)
ax.plot(x_arr,nums_month_merra2_fixed_poleward,color='deepskyblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1957.5,2023.5,1./12.)
ax.plot(x_arr,nums_month_jra55_polar,color='r',linewidth=width,alpha=0.5)
ax.plot(x_arr,nums_month_jra55_fixed,color='gold',linewidth=width,alpha=0.5)
ax.plot(x_arr,nums_month_jra55_fixed_poleward,color='royalblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1939.5,2023.5,1./12.)
ax.plot(x_arr,nums_month_era5_polar,color='darkred',linewidth=width,alpha=0.5)
ax.plot(x_arr,nums_month_era5_fixed,color='orange',linewidth=width,alpha=0.5)
ax.plot(x_arr,nums_month_era5_fixed_poleward,color='mediumblue',linewidth=width,alpha=0.5)

ax.fill_between([1939,1979.5],7.5,18,color='gray',alpha=0.4)

width=8

ax.plot(yrs1,[x/1. for x in nums_list_era5_polar],color='darkred',linewidth=width,label=label_5,zorder=10)
ax.plot(yrs2,[x/1. for x in nums_list_jra55_polar],color='r',linewidth=width,label=label_8,zorder=13)
ax.plot(yrs,[x/1. for x in nums_list_merra2_polar],color='salmon',linewidth=width,label=label_2,zorder=16)

ax.plot(yrs1,[x/1. for x in nums_list_era5_fixed],color='orange',linewidth=width,label=label_4,zorder=11)
ax.plot(yrs2,[x/1. for x in nums_list_jra55_fixed],color='gold',linewidth=width,label=label_7,zorder=14)
ax.plot(yrs,[x/1. for x in nums_list_merra2_fixed],color='yellow',linewidth=width,label=label_1,zorder=17)

ax.plot(yrs1,[x/1. for x in nums_list_era5_fixed_poleward],color='mediumblue',linewidth=width,label=label_6,zorder=12)
ax.plot(yrs2,[x/1. for x in nums_list_jra55_fixed_poleward],color='royalblue',linewidth=width,label=label_9,zorder=15)
ax.plot(yrs,[x/1. for x in nums_list_merra2_fixed_poleward],color='deepskyblue',linewidth=width,label=label_3,zorder=18)

width=2.5
yrs=np.arange(2000,2024,1)
test=nums_list_merra2_polar[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=nums_list_merra2_fixed[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=nums_list_merra2_fixed_poleward[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=nums_list_era5_polar[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=nums_list_era5_fixed[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=nums_list_era5_fixed_poleward[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=nums_list_jra55_polar[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=nums_list_jra55_fixed[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=nums_list_jra55_fixed_poleward[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


y=np.arange(0,25,0.1)

ax.tick_params(labelsize=22)
ax.set_xlabel('Year',fontsize=24)
ax.set_ylabel('Mean # of ARs globally',fontsize=24)
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{number}$',fontsize=36,pad=5)
ax.grid(True)
plt.legend(loc='upper center',fontsize=16.75,ncol=1)
ax.set_xlim(1939,2024.2)
ax.set_ylim(7.5,18)
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.5)
    
plt.tight_layout()
plt.show()

In [ ]:
fig.savefig(dir_data+'ar_lines_plots_new_1.png')

In [ ]:
yrs=np.arange(1940,2024,1)
for i in range(len(yrs)):
    if i==0:
        nums_month_era5_polar=list(np.asarray(areas_array_era5_polar)[:,i])
        nums_month_era5_fixed=list(np.asarray(areas_array_era5_fixed)[:,i])
        nums_month_era5_fixed_poleward=list(np.asarray(areas_array_era5_fixed_poleward)[:,i])
        
    else:
        nums_month_era5_polar=nums_month_era5_polar+list(np.asarray(areas_array_era5_polar)[:,i])
        nums_month_era5_fixed=nums_month_era5_fixed+list(np.asarray(areas_array_era5_fixed)[:,i])
        nums_month_era5_fixed_poleward=nums_month_era5_fixed_poleward+list(np.asarray(areas_array_era5_fixed_poleward)[:,i])

yrs=np.arange(1980,2024,1)
for i in range(len(yrs)):
    if i==0:
        nums_month_merra2_polar=list(np.asarray(areas_array_merra2_polar)[:,i-1])
        nums_month_merra2_fixed=list(np.asarray(areas_array_merra2_fixed)[:,i-1])
        nums_month_merra2_fixed_poleward=list(np.asarray(areas_array_merra2_fixed_poleward)[:,i-1])
        
    else:
        nums_month_merra2_polar=nums_month_merra2_polar+list(np.asarray(areas_array_merra2_polar)[:,i-1])
        nums_month_merra2_fixed=nums_month_merra2_fixed+list(np.asarray(areas_array_merra2_fixed)[:,i-1])
        nums_month_merra2_fixed_poleward=nums_month_merra2_fixed_poleward+list(np.asarray(areas_array_merra2_fixed_poleward)[:,i-1])

yrs=np.arange(1958,2024,1)
for i in range(len(yrs)):
    if i==0:
        nums_month_jra55_polar=list(np.asarray(areas_array_jra55_polar)[:,i-1])
        nums_month_jra55_fixed=list(np.asarray(areas_array_jra55_fixed)[:,i-1])
        nums_month_jra55_fixed_poleward=list(np.asarray(areas_array_jra55_fixed_poleward)[:,i-1])
        
    else:
        nums_month_jra55_polar=nums_month_jra55_polar+list(np.asarray(areas_array_jra55_polar)[:,i-1])
        nums_month_jra55_fixed=nums_month_jra55_fixed+list(np.asarray(areas_array_jra55_fixed)[:,i-1])
        nums_month_jra55_fixed_poleward=nums_month_jra55_fixed_poleward+list(np.asarray(areas_array_jra55_fixed_poleward)[:,i-1])

        
x_arr=np.arange(1979.5,2023.5,1./12.)


month_days=[31,28.25,31,30,31,30,31,31,30,31,30,31]

sigs_list=[]

yrs=np.arange(1980,2024,1)
test=[x/1e6 for x in areas_list_merra2_fixed]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e6 for x in areas_list_merra2_polar]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='MERRA-2 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e6 for x in areas_list_merra2_fixed_poleward]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='MERRA-2 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs1=np.arange(1940+40,2024,1)
test=[x/1e6 for x in areas_list_era5_fixed[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_4='ERA5 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e6 for x in areas_list_era5_polar[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_5='ERA5 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e6 for x in areas_list_era5_fixed_poleward[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_6='ERA5 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs2=np.arange(1958+22,2024,1)
test=[x/1e6 for x in areas_list_jra55_fixed[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_7='JRA-55 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e6 for x in areas_list_jra55_polar[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_8='JRA-55 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e6 for x in areas_list_jra55_fixed_poleward[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_9='JRA-55 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'


yrs1=np.arange(1940,2024,1)
yrs2=np.arange(1958,2024,1)

fig=plt.figure(figsize=(9,15))
ax=plt.subplot(1,1,1)

#MERRA-2: polar=salmon, fixed = yellow, fixed poleward = deepskyblue
#JRA-55: polar = r, fixed = gold, fixed poleward = royalblue
#ERA5: polar = darkred, fixed = orange, fixed poleward = mediumblue

width=0.75
ax.plot(x_arr,[x/1e6 for x in nums_month_merra2_polar],color='salmon',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e6 for x in nums_month_merra2_fixed],color='yellow',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e6 for x in nums_month_merra2_fixed_poleward],color='deepskyblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1957.5,2023.5,1./12.)
ax.plot(x_arr,[x/1e6 for x in nums_month_jra55_polar],color='r',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e6 for x in nums_month_jra55_fixed],color='gold',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e6 for x in nums_month_jra55_fixed_poleward],color='royalblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1939.5,2023.5,1./12.)
ax.plot(x_arr,[x/1e6 for x in nums_month_era5_polar],color='darkred',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e6 for x in nums_month_era5_fixed],color='orange',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e6 for x in nums_month_era5_fixed_poleward],color='mediumblue',linewidth=width,alpha=0.5)

ax.fill_between([1939,1979.5],2.1,5.2,color='gray',alpha=0.4)

width=8

ax.plot(yrs1,[x/1e6 for x in areas_list_era5_polar],color='darkred',linewidth=width,label=label_5,zorder=10)
ax.plot(yrs2,[x/1e6 for x in areas_list_jra55_polar],color='r',linewidth=width,label=label_8,zorder=13)
ax.plot(yrs,[x/1e6 for x in areas_list_merra2_polar],color='salmon',linewidth=width,label=label_2,zorder=16)

ax.plot(yrs1,[x/1e6 for x in areas_list_era5_fixed],color='orange',linewidth=width,label=label_4,zorder=11)
ax.plot(yrs2,[x/1e6 for x in areas_list_jra55_fixed],color='gold',linewidth=width,label=label_7,zorder=14)
ax.plot(yrs,[x/1e6 for x in areas_list_merra2_fixed],color='yellow',linewidth=width,label=label_1,zorder=17)

ax.plot(yrs1,[x/1e6 for x in areas_list_era5_fixed_poleward],color='mediumblue',linewidth=width,label=label_6,zorder=12)
ax.plot(yrs2,[x/1e6 for x in areas_list_jra55_fixed_poleward],color='royalblue',linewidth=width,label=label_9,zorder=15)
ax.plot(yrs,[x/1e6 for x in areas_list_merra2_fixed_poleward],color='deepskyblue',linewidth=width,label=label_3,zorder=18)

width=2.5
yrs=np.arange(2000,2024,1)
test=[x/1e6 for x in areas_list_merra2_polar[-24:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e6 for x in areas_list_merra2_fixed[-24:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e6 for x in areas_list_merra2_fixed_poleward[-24:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=[x/1e6 for x in areas_list_era5_polar[-24:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e6 for x in areas_list_era5_fixed[-24:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e6 for x in areas_list_era5_fixed_poleward[-24:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=[x/1e6 for x in areas_list_jra55_polar[-24:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e6 for x in areas_list_jra55_fixed[-24:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e6 for x in areas_list_jra55_fixed_poleward[-24:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


y=np.arange(0,25,0.1)

ax.tick_params(labelsize=22)
ax.set_xlabel('Year',fontsize=24)
ax.set_ylabel('10$^{6}$ km$^{2}$',fontsize=24)
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{area}$',fontsize=36,pad=5)
ax.grid(True)
plt.legend(loc='upper center',fontsize=16.75,ncol=1)
ax.set_xlim(1939,2024.2)
ax.set_ylim(2.1,5.2)
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.5)
    
plt.tight_layout()
plt.show()

In [ ]:
fig.savefig(dir_data+'ar_lines_plots_new_2.png')

In [ ]:
yrs=np.arange(1940,2024,1)
for i in range(len(yrs)):
    if i==0:
        nums_month_era5_polar=list(np.asarray(lats_array_era5_polar)[:,i])
        nums_month_era5_fixed=list(np.asarray(lats_array_era5_fixed)[:,i])
        nums_month_era5_fixed_poleward=list(np.asarray(lats_array_era5_fixed_poleward)[:,i])
        
    else:
        nums_month_era5_polar=nums_month_era5_polar+list(np.asarray(lats_array_era5_polar)[:,i])
        nums_month_era5_fixed=nums_month_era5_fixed+list(np.asarray(lats_array_era5_fixed)[:,i])
        nums_month_era5_fixed_poleward=nums_month_era5_fixed_poleward+list(np.asarray(lats_array_era5_fixed_poleward)[:,i])

yrs=np.arange(1980,2024,1)
for i in range(len(yrs)):
    if i==0:
        nums_month_merra2_polar=list(np.asarray(lats_array_merra2_polar)[:,i-1])
        nums_month_merra2_fixed=list(np.asarray(lats_array_merra2_fixed)[:,i-1])
        nums_month_merra2_fixed_poleward=list(np.asarray(lats_array_merra2_fixed_poleward)[:,i-1])
        
    else:
        nums_month_merra2_polar=nums_month_merra2_polar+list(np.asarray(lats_array_merra2_polar)[:,i-1])
        nums_month_merra2_fixed=nums_month_merra2_fixed+list(np.asarray(lats_array_merra2_fixed)[:,i-1])
        nums_month_merra2_fixed_poleward=nums_month_merra2_fixed_poleward+list(np.asarray(lats_array_merra2_fixed_poleward)[:,i-1])

yrs=np.arange(1958,2024,1)
for i in range(len(yrs)):
    if i==0:
        nums_month_jra55_polar=list(np.asarray(lats_array_jra55_polar)[:,i-1])
        nums_month_jra55_fixed=list(np.asarray(lats_array_jra55_fixed)[:,i-1])
        nums_month_jra55_fixed_poleward=list(np.asarray(lats_array_jra55_fixed_poleward)[:,i-1])
        
    else:
        nums_month_jra55_polar=nums_month_jra55_polar+list(np.asarray(lats_array_jra55_polar)[:,i-1])
        nums_month_jra55_fixed=nums_month_jra55_fixed+list(np.asarray(lats_array_jra55_fixed)[:,i-1])
        nums_month_jra55_fixed_poleward=nums_month_jra55_fixed_poleward+list(np.asarray(lats_array_jra55_fixed_poleward)[:,i-1])

        
x_arr=np.arange(1979.5,2023.5,1./12.)


month_days=[31,28.25,31,30,31,30,31,31,30,31,30,31]

sigs_list=[]

yrs=np.arange(1980,2024,1)
test=[x/1. for x in lats_list_merra2_fixed]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_merra2_polar]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='MERRA-2 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_merra2_fixed_poleward]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='MERRA-2 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs1=np.arange(1940+40,2024,1)
test=[x/1. for x in lats_list_era5_fixed[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_4='ERA5 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_era5_polar[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_5='ERA5 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_era5_fixed_poleward[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_6='ERA5 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs2=np.arange(1958+22,2024,1)
test=[x/1. for x in lats_list_jra55_fixed[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_7='JRA-55 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_jra55_polar[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_8='JRA-55 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_jra55_fixed_poleward[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_9='JRA-55 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'


yrs1=np.arange(1940,2024,1)
yrs2=np.arange(1958,2024,1)

fig=plt.figure(figsize=(9,15))
ax=plt.subplot(1,1,1)

#MERRA-2: polar=salmon, fixed = yellow, fixed poleward = deepskyblue
#JRA-55: polar = r, fixed = gold, fixed poleward = royalblue
#ERA5: polar = darkred, fixed = orange, fixed poleward = mediumblue

width=0.75
ax.plot(x_arr,nums_month_merra2_polar,color='salmon',linewidth=width,alpha=0.5)
ax.plot(x_arr,nums_month_merra2_fixed,color='yellow',linewidth=width,alpha=0.5)
ax.plot(x_arr,nums_month_merra2_fixed_poleward,color='deepskyblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1957.5,2023.5,1./12.)
ax.plot(x_arr,nums_month_jra55_polar,color='r',linewidth=width,alpha=0.5)
ax.plot(x_arr,nums_month_jra55_fixed,color='gold',linewidth=width,alpha=0.5)
ax.plot(x_arr,nums_month_jra55_fixed_poleward,color='royalblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1939.5,2023.5,1./12.)
ax.plot(x_arr,nums_month_era5_polar,color='darkred',linewidth=width,alpha=0.5)
ax.plot(x_arr,nums_month_era5_fixed,color='orange',linewidth=width,alpha=0.5)
ax.plot(x_arr,nums_month_era5_fixed_poleward,color='mediumblue',linewidth=width,alpha=0.5)

ax.fill_between([1939,1979.5],37.5,50,color='gray',alpha=0.4)

width=8

ax.plot(yrs1,[x/1. for x in lats_list_era5_polar],color='darkred',linewidth=width,label=label_5,zorder=10)
ax.plot(yrs2,[x/1. for x in lats_list_jra55_polar],color='r',linewidth=width,label=label_8,zorder=13)
ax.plot(yrs,[x/1. for x in lats_list_merra2_polar],color='salmon',linewidth=width,label=label_2,zorder=16)

ax.plot(yrs1,[x/1. for x in lats_list_era5_fixed],color='orange',linewidth=width,label=label_4,zorder=11)
ax.plot(yrs2,[x/1. for x in lats_list_jra55_fixed],color='gold',linewidth=width,label=label_7,zorder=14)
ax.plot(yrs,[x/1. for x in lats_list_merra2_fixed],color='yellow',linewidth=width,label=label_1,zorder=17)

ax.plot(yrs1,[x/1. for x in lats_list_era5_fixed_poleward],color='mediumblue',linewidth=width,label=label_6,zorder=12)
ax.plot(yrs2,[x/1. for x in lats_list_jra55_fixed_poleward],color='royalblue',linewidth=width,label=label_9,zorder=15)
ax.plot(yrs,[x/1. for x in lats_list_merra2_fixed_poleward],color='deepskyblue',linewidth=width,label=label_3,zorder=18)

width=2.5
yrs=np.arange(2000,2024,1)
test=lats_list_merra2_polar[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_merra2_fixed[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_merra2_fixed_poleward[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=lats_list_era5_polar[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_era5_fixed[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_era5_fixed_poleward[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=lats_list_jra55_polar[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_jra55_fixed[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_jra55_fixed_poleward[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


y=np.arange(0,25,0.1)

ax.tick_params(labelsize=22)
ax.set_xlabel('Year',fontsize=24)
ax.set_ylabel('Degrees N/S',fontsize=24)
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{mean}$'+' '+r'$\bf{latitude}$',fontsize=36,pad=5)
ax.grid(True)
plt.legend(loc='upper center',fontsize=16.75,ncol=1)
ax.set_xlim(1939,2024.2)
ax.set_ylim(37.5,50)
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.5)
    
plt.tight_layout()
plt.show()

In [ ]:
fig.savefig(dir_data+'ar_lines_plots_new_3.png')

In [ ]:
yrs=np.arange(1940,2024,1)
for i in range(len(yrs)):
    if i==0:
        nums_month_era5_polar=list(np.asarray(nums_array_era5_polar)[:,i])
        nums_month_era5_fixed=list(np.asarray(nums_array_era5_fixed)[:,i])
        nums_month_era5_fixed_poleward=list(np.asarray(nums_array_era5_fixed_poleward)[:,i])
        
    else:
        nums_month_era5_polar=nums_month_era5_polar+list(np.asarray(nums_array_era5_polar)[:,i])
        nums_month_era5_fixed=nums_month_era5_fixed+list(np.asarray(nums_array_era5_fixed)[:,i])
        nums_month_era5_fixed_poleward=nums_month_era5_fixed_poleward+list(np.asarray(nums_array_era5_fixed_poleward)[:,i])

yrs=np.arange(1980,2024,1)
for i in range(len(yrs)):
    if i==0:
        nums_month_merra2_polar=list(np.asarray(nums_array_merra2_polar)[:,i-1])
        nums_month_merra2_fixed=list(np.asarray(nums_array_merra2_fixed)[:,i-1])
        nums_month_merra2_fixed_poleward=list(np.asarray(nums_array_merra2_fixed_poleward)[:,i-1])
        
    else:
        nums_month_merra2_polar=nums_month_merra2_polar+list(np.asarray(nums_array_merra2_polar)[:,i-1])
        nums_month_merra2_fixed=nums_month_merra2_fixed+list(np.asarray(nums_array_merra2_fixed)[:,i-1])
        nums_month_merra2_fixed_poleward=nums_month_merra2_fixed_poleward+list(np.asarray(nums_array_merra2_fixed_poleward)[:,i-1])

yrs=np.arange(1958,2024,1)
for i in range(len(yrs)):
    if i==0:
        nums_month_jra55_polar=list(np.asarray(nums_array_jra55_polar)[:,i-1])
        nums_month_jra55_fixed=list(np.asarray(nums_array_jra55_fixed)[:,i-1])
        nums_month_jra55_fixed_poleward=list(np.asarray(nums_array_jra55_fixed_poleward)[:,i-1])
        
    else:
        nums_month_jra55_polar=nums_month_jra55_polar+list(np.asarray(nums_array_jra55_polar)[:,i-1])
        nums_month_jra55_fixed=nums_month_jra55_fixed+list(np.asarray(nums_array_jra55_fixed)[:,i-1])
        nums_month_jra55_fixed_poleward=nums_month_jra55_fixed_poleward+list(np.asarray(nums_array_jra55_fixed_poleward)[:,i-1])

        
x_arr=np.arange(1979.5,2023.5,1./12.)

sigs_list=[]

month_days=[31,28.25,31,30,31,30,31,31,30,31,30,31]


yrs=np.arange(1980,2024,1)
test=[x/1. for x in nums_list_merra2_fixed]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in nums_list_merra2_polar]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='MERRA-2 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in nums_list_merra2_fixed_poleward]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='MERRA-2 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs1=np.arange(1940+40,2024,1)
test=[x/1. for x in nums_list_era5_fixed[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_4='ERA5 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in nums_list_era5_polar[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_5='ERA5 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in nums_list_era5_fixed_poleward[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_6='ERA5 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs2=np.arange(1958+22,2024,1)
test=[x/1. for x in nums_list_jra55_fixed[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_7='JRA-55 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in nums_list_jra55_polar[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_8='JRA-55 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in nums_list_jra55_fixed_poleward[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_9='JRA-55 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'


yrs1=np.arange(1940,2024,1)
yrs2=np.arange(1958,2024,1)

fig=plt.figure(figsize=(15,15))
ax=plt.subplot(1,1,1)

#MERRA-2: polar=salmon, fixed = yellow, fixed poleward = deepskyblue
#JRA-55: polar = r, fixed = gold, fixed poleward = royalblue
#ERA5: polar = darkred, fixed = orange, fixed poleward = mediumblue

test1=[x-0.5 for x in nums_list_merra2_fixed]
test2=[x-0.5 for x in nums_list_merra2_fixed_poleward]
test3=[x-0.5 for x in nums_list_merra2_polar]

width=0.75
ax.plot(x_arr,nums_month_merra2_polar,color='salmon',linewidth=width,alpha=0.5)
ax.plot(x_arr,nums_month_merra2_fixed,color='yellow',linewidth=width,alpha=0.5)
ax.plot(x_arr,nums_month_merra2_fixed_poleward,color='deepskyblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1957.5,2023.5,1./12.)
ax.plot(x_arr,nums_month_jra55_polar,color='r',linewidth=width,alpha=0.5)
ax.plot(x_arr,nums_month_jra55_fixed,color='gold',linewidth=width,alpha=0.5)
ax.plot(x_arr,nums_month_jra55_fixed_poleward,color='royalblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1939.5,2023.5,1./12.)
ax.plot(x_arr,nums_month_era5_polar,color='darkred',linewidth=width,alpha=0.5)
ax.plot(x_arr,nums_month_era5_fixed,color='orange',linewidth=width,alpha=0.5)
ax.plot(x_arr,nums_month_era5_fixed_poleward,color='mediumblue',linewidth=width,alpha=0.5)

ax.fill_between([1939,1979.5],7.5,16.5,color='gray',alpha=0.4)

width=8

ax.plot(yrs1,[x/1. for x in nums_list_era5_polar],color='darkred',linewidth=width,label=label_5,zorder=10)
ax.plot(yrs2,[x/1. for x in nums_list_jra55_polar],color='r',linewidth=width,label=label_8,zorder=13)
ax.plot(yrs,[x/1. for x in nums_list_merra2_polar],color='salmon',linewidth=width,label=label_2,zorder=16)

ax.plot(yrs1,[x/1. for x in nums_list_era5_fixed],color='orange',linewidth=width,label=label_4,zorder=11)
ax.plot(yrs2,[x/1. for x in nums_list_jra55_fixed],color='gold',linewidth=width,label=label_7,zorder=14)
ax.plot(yrs,[x/1. for x in nums_list_merra2_fixed],color='yellow',linewidth=width,label=label_1,zorder=17)

ax.plot(yrs1,[x/1. for x in nums_list_era5_fixed_poleward],color='mediumblue',linewidth=width,label=label_6,zorder=12)
ax.plot(yrs2,[x/1. for x in nums_list_jra55_fixed_poleward],color='royalblue',linewidth=width,label=label_9,zorder=15)
ax.plot(yrs,[x/1. for x in nums_list_merra2_fixed_poleward],color='deepskyblue',linewidth=width,label=label_3,zorder=18)

width=2.5
yrs=np.arange(2000,2024,1)
test=nums_list_merra2_polar[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=nums_list_merra2_fixed[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=nums_list_merra2_fixed_poleward[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=nums_list_era5_polar[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=nums_list_era5_fixed[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=nums_list_era5_fixed_poleward[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=nums_list_jra55_polar[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=nums_list_jra55_fixed[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=nums_list_jra55_fixed_poleward[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


y=np.arange(0,25,0.1)

ax.tick_params(labelsize=22)
ax.set_xlabel('Year',fontsize=24)
ax.set_ylabel('Mean # of ARs globally',fontsize=24)
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{number}$',fontsize=36,pad=5)
ax.grid(True)
plt.legend(loc='upper center',fontsize=13.8,ncol=2)
ax.set_xlim(1939.3,2023.6)
ax.set_ylim(7.5,16.5)
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.5)
    
plt.show()

In [ ]:
fig.savefig(dir_data+'ar_obs_lines_1.png',dpi=300)

In [ ]:
yrs=np.arange(1940,2024,1)
for i in range(len(yrs)):
    if i==0:
        areas_month_era5_polar=list(np.asarray(areas_array_era5_polar)[:,i])
        areas_month_era5_fixed=list(np.asarray(areas_array_era5_fixed)[:,i])
        areas_month_era5_fixed_poleward=list(np.asarray(areas_array_era5_fixed_poleward)[:,i])
        
    else:
        areas_month_era5_polar=areas_month_era5_polar+list(np.asarray(areas_array_era5_polar)[:,i])
        areas_month_era5_fixed=areas_month_era5_fixed+list(np.asarray(areas_array_era5_fixed)[:,i])
        areas_month_era5_fixed_poleward=areas_month_era5_fixed_poleward+list(np.asarray(areas_array_era5_fixed_poleward)[:,i])

yrs=np.arange(1980,2024,1)
for i in range(len(yrs)):
    if i==0:
        areas_month_merra2_polar=list(np.asarray(areas_array_merra2_polar)[:,i-1])
        areas_month_merra2_fixed=list(np.asarray(areas_array_merra2_fixed)[:,i-1])
        areas_month_merra2_fixed_poleward=list(np.asarray(areas_array_merra2_fixed_poleward)[:,i-1])
        
    else:
        areas_month_merra2_polar=areas_month_merra2_polar+list(np.asarray(areas_array_merra2_polar)[:,i-1])
        areas_month_merra2_fixed=areas_month_merra2_fixed+list(np.asarray(areas_array_merra2_fixed)[:,i-1])
        areas_month_merra2_fixed_poleward=areas_month_merra2_fixed_poleward+list(np.asarray(areas_array_merra2_fixed_poleward)[:,i-1])

yrs=np.arange(1958,2024,1)
for i in range(len(yrs)):
    if i==0:
        areas_month_jra55_polar=list(np.asarray(areas_array_jra55_polar)[:,i-1])
        areas_month_jra55_fixed=list(np.asarray(areas_array_jra55_fixed)[:,i-1])
        areas_month_jra55_fixed_poleward=list(np.asarray(areas_array_jra55_fixed_poleward)[:,i-1])
        
    else:
        areas_month_jra55_polar=areas_month_jra55_polar+list(np.asarray(areas_array_jra55_polar)[:,i-1])
        areas_month_jra55_fixed=areas_month_jra55_fixed+list(np.asarray(areas_array_jra55_fixed)[:,i-1])
        areas_month_jra55_fixed_poleward=areas_month_jra55_fixed_poleward+list(np.asarray(areas_array_jra55_fixed_poleward)[:,i-1])

        
x_arr=np.arange(1979.5,2023.5,1./12.)


month_days=[31,28.25,31,30,31,30,31,31,30,31,30,31]


yrs=np.arange(1980,2024,1)
test=[x/1e6 for x in areas_list_merra2_fixed]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e6 for x in areas_list_merra2_polar]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='MERRA-2 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e6 for x in areas_list_merra2_fixed_poleward]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='MERRA-2 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs1=np.arange(1940+40,2024,1)
test=[x/1e6 for x in areas_list_era5_fixed[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_4='ERA5 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e6 for x in areas_list_era5_polar[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_5='ERA5 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e6 for x in areas_list_era5_fixed_poleward[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_6='ERA5 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs2=np.arange(1958+22,2024,1)
test=[x/1e6 for x in areas_list_jra55_fixed[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_7='JRA-55 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e6 for x in areas_list_jra55_polar[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_8='JRA-55 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1e6 for x in areas_list_jra55_fixed_poleward[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_9='JRA-55 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'


yrs1=np.arange(1940,2024,1)
yrs2=np.arange(1958,2024,1)

fig=plt.figure(figsize=(15,15))
ax=plt.subplot(1,1,1)

#MERRA-2: polar=salmon, fixed = yellow, fixed poleward = deepskyblue
#JRA-55: polar = r, fixed = gold, fixed poleward = royalblue
#ERA5: polar = darkred, fixed = orange, fixed poleward = mediumblue

width=0.75
ax.plot(x_arr,[x/1e6 for x in areas_month_merra2_polar],color='salmon',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e6 for x in areas_month_merra2_fixed],color='yellow',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e6 for x in areas_month_merra2_fixed_poleward],color='deepskyblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1957.5,2023.5,1./12.)
ax.plot(x_arr,[x/1e6 for x in areas_month_jra55_polar],color='r',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e6 for x in areas_month_jra55_fixed],color='gold',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e6 for x in areas_month_jra55_fixed_poleward],color='royalblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1939.5,2023.5,1./12.)
ax.plot(x_arr,[x/1e6 for x in areas_month_era5_polar],color='darkred',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e6 for x in areas_month_era5_fixed],color='orange',linewidth=width,alpha=0.5)
ax.plot(x_arr,[x/1e6 for x in areas_month_era5_fixed_poleward],color='mediumblue',linewidth=width,alpha=0.5)

ax.fill_between([1939,1979.5],1.95,4.7,color='gray',alpha=0.4)

width=8

ax.plot(yrs1,[x/1e6 for x in areas_list_era5_polar],color='darkred',linewidth=width,label=label_5,zorder=10)
ax.plot(yrs2,[x/1e6 for x in areas_list_jra55_polar],color='r',linewidth=width,label=label_8,zorder=13)
ax.plot(yrs,[x/1e6 for x in areas_list_merra2_polar],color='salmon',linewidth=width,label=label_2,zorder=16)

ax.plot(yrs1,[x/1e6 for x in areas_list_era5_fixed],color='orange',linewidth=width,label=label_4,zorder=11)
ax.plot(yrs2,[x/1e6 for x in areas_list_jra55_fixed],color='gold',linewidth=width,label=label_7,zorder=14)
ax.plot(yrs,[x/1e6 for x in areas_list_merra2_fixed],color='yellow',linewidth=width,label=label_1,zorder=17)

ax.plot(yrs1,[x/1e6 for x in areas_list_era5_fixed_poleward],color='mediumblue',linewidth=width,label=label_6,zorder=12)
ax.plot(yrs2,[x/1e6 for x in areas_list_jra55_fixed_poleward],color='royalblue',linewidth=width,label=label_9,zorder=15)
ax.plot(yrs,[x/1e6 for x in areas_list_merra2_fixed_poleward],color='deepskyblue',linewidth=width,label=label_3,zorder=18)

width=2.5
yrs=np.arange(2000,2024,1)
test=[x/1e6 for x in areas_list_merra2_polar[-24:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e6 for x in areas_list_merra2_fixed[-24:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e6 for x in areas_list_merra2_fixed_poleward[-24:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=[x/1e6 for x in areas_list_era5_polar[-24:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e6 for x in areas_list_era5_fixed[-24:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e6 for x in areas_list_era5_fixed_poleward[-24:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=[x/1e6 for x in areas_list_jra55_polar[-24:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e6 for x in areas_list_jra55_fixed[-24:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=[x/1e6 for x in areas_list_jra55_fixed_poleward[-24:]]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


y=np.arange(0,25,0.1)

ax.tick_params(labelsize=22)
ax.set_xlabel('Year',fontsize=24)
ax.set_ylabel('10$^{6}$ km$^{2}$',fontsize=24)
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{area}$',fontsize=36,pad=5)
ax.grid(True)
plt.legend(loc='upper center',fontsize=14.1,ncol=2)
ax.set_xlim(1939.3,2023.6)
ax.set_ylim(1.95,4.7)
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.5)
    
plt.show()

In [ ]:
fig.savefig(dir_data+'ar_obs_lines_2.png',dpi=300)

In [ ]:
yrs=np.arange(1940,2024,1)
for i in range(len(yrs)):
    if i==0:
        lats_month_era5_polar=list(np.asarray(lats_array_era5_polar)[:,i])
        lats_month_era5_fixed=list(np.asarray(lats_array_era5_fixed)[:,i])
        lats_month_era5_fixed_poleward=list(np.asarray(lats_array_era5_fixed_poleward)[:,i])
        
    else:
        lats_month_era5_polar=lats_month_era5_polar+list(np.asarray(lats_array_era5_polar)[:,i])
        lats_month_era5_fixed=lats_month_era5_fixed+list(np.asarray(lats_array_era5_fixed)[:,i])
        lats_month_era5_fixed_poleward=lats_month_era5_fixed_poleward+list(np.asarray(lats_array_era5_fixed_poleward)[:,i])

yrs=np.arange(1980,2024,1)
for i in range(len(yrs)):
    if i==0:
        lats_month_merra2_polar=list(np.asarray(lats_array_merra2_polar)[:,i-1])
        lats_month_merra2_fixed=list(np.asarray(lats_array_merra2_fixed)[:,i-1])
        lats_month_merra2_fixed_poleward=list(np.asarray(lats_array_merra2_fixed_poleward)[:,i-1])
        
    else:
        lats_month_merra2_polar=lats_month_merra2_polar+list(np.asarray(lats_array_merra2_polar)[:,i-1])
        lats_month_merra2_fixed=lats_month_merra2_fixed+list(np.asarray(lats_array_merra2_fixed)[:,i-1])
        lats_month_merra2_fixed_poleward=lats_month_merra2_fixed_poleward+list(np.asarray(lats_array_merra2_fixed_poleward)[:,i-1])

yrs=np.arange(1958,2024,1)
for i in range(len(yrs)):
    if i==0:
        lats_month_jra55_polar=list(np.asarray(lats_array_jra55_polar)[:,i-1])
        lats_month_jra55_fixed=list(np.asarray(lats_array_jra55_fixed)[:,i-1])
        lats_month_jra55_fixed_poleward=list(np.asarray(lats_array_jra55_fixed_poleward)[:,i-1])
        
    else:
        lats_month_jra55_polar=lats_month_jra55_polar+list(np.asarray(lats_array_jra55_polar)[:,i-1])
        lats_month_jra55_fixed=lats_month_jra55_fixed+list(np.asarray(lats_array_jra55_fixed)[:,i-1])
        lats_month_jra55_fixed_poleward=lats_month_jra55_fixed_poleward+list(np.asarray(lats_array_jra55_fixed_poleward)[:,i-1])

        
x_arr=np.arange(1979.5,2023.5,1./12.)


month_days=[31,28.25,31,30,31,30,31,31,30,31,30,31]


yrs=np.arange(1980,2024,1)
test=[x/1. for x in lats_list_merra2_fixed]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_merra2_polar]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='MERRA-2 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_merra2_fixed_poleward]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='MERRA-2 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs1=np.arange(1940+40,2024,1)
test=[x/1. for x in lats_list_era5_fixed[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_4='ERA5 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_era5_polar[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_5='ERA5 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_era5_fixed_poleward[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_6='ERA5 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs2=np.arange(1958+22,2024,1)
test=[x/1. for x in lats_list_jra55_fixed[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_7='JRA-55 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_jra55_polar[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_8='JRA-55 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_jra55_fixed_poleward[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_9='JRA-55 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'


yrs1=np.arange(1940,2024,1)
yrs2=np.arange(1958,2024,1)

fig=plt.figure(figsize=(15,15))
ax=plt.subplot(1,1,1)

#MERRA-2: polar=salmon, fixed = yellow, fixed poleward = deepskyblue
#JRA-55: polar = r, fixed = gold, fixed poleward = royalblue
#ERA5: polar = darkred, fixed = orange, fixed poleward = mediumblue

width=0.75
ax.plot(x_arr,lats_month_merra2_polar,color='salmon',linewidth=width,alpha=0.5)
ax.plot(x_arr,lats_month_merra2_fixed,color='yellow',linewidth=width,alpha=0.5)
ax.plot(x_arr,lats_month_merra2_fixed_poleward,color='deepskyblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1957.5,2023.5,1./12.)
ax.plot(x_arr,lats_month_jra55_polar,color='r',linewidth=width,alpha=0.5)
ax.plot(x_arr,lats_month_jra55_fixed,color='gold',linewidth=width,alpha=0.5)
ax.plot(x_arr,lats_month_jra55_fixed_poleward,color='royalblue',linewidth=width,alpha=0.5)

x_arr=np.arange(1939.5,2023.5,1./12.)
ax.plot(x_arr,lats_month_era5_polar,color='darkred',linewidth=width,alpha=0.5)
ax.plot(x_arr,lats_month_era5_fixed,color='orange',linewidth=width,alpha=0.5)
ax.plot(x_arr,lats_month_era5_fixed_poleward,color='mediumblue',linewidth=width,alpha=0.5)

ax.fill_between([1939,1979.5],37,49,color='gray',alpha=0.4)

width=8

ax.plot(yrs1,[x/1. for x in lats_list_era5_polar],color='darkred',linewidth=width,label=label_5,zorder=10)
ax.plot(yrs2,[x/1. for x in lats_list_jra55_polar],color='r',linewidth=width,label=label_8,zorder=13)
ax.plot(yrs,[x/1. for x in lats_list_merra2_polar],color='salmon',linewidth=width,label=label_2,zorder=16)

ax.plot(yrs1,[x/1. for x in lats_list_era5_fixed],color='orange',linewidth=width,label=label_4,zorder=11)
ax.plot(yrs2,[x/1. for x in lats_list_jra55_fixed],color='gold',linewidth=width,label=label_7,zorder=14)
ax.plot(yrs,[x/1. for x in lats_list_merra2_fixed],color='yellow',linewidth=width,label=label_1,zorder=17)

ax.plot(yrs1,[x/1. for x in lats_list_era5_fixed_poleward],color='mediumblue',linewidth=width,label=label_6,zorder=12)
ax.plot(yrs2,[x/1. for x in lats_list_jra55_fixed_poleward],color='royalblue',linewidth=width,label=label_9,zorder=15)
ax.plot(yrs,[x/1. for x in lats_list_merra2_fixed_poleward],color='deepskyblue',linewidth=width,label=label_3,zorder=18)

width=2.5
yrs=np.arange(2000,2024,1)
test=lats_list_merra2_polar[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_merra2_fixed[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_merra2_fixed_poleward[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=lats_list_era5_polar[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_era5_fixed[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_era5_fixed_poleward[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=lats_list_jra55_polar[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_jra55_fixed[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_jra55_fixed_poleward[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


y=np.arange(0,25,0.1)

ax.tick_params(labelsize=22)
ax.set_xlabel('Year',fontsize=24)
ax.set_ylabel('Degrees N/S',fontsize=24)
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{mean}$'+' '+r'$\bf{latitude}$',fontsize=36,pad=5)
ax.grid(True)
plt.legend(loc='upper center',fontsize=13.8,ncol=2)
ax.set_xlim(1939.3,2023.6)
ax.set_ylim(37,49)
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.5)
    
plt.show()

In [ ]:
fig.savefig(dir_data+'ar_obs_lines_3.png',dpi=300)

In [ ]:
yrs=np.arange(1940,2024,1)
for i in range(len(yrs)):
    if i==0:
        lats_month_era5_polar=list(np.asarray(lats_array_era5_polar_nh)[:,i])
        lats_month_era5_fixed=list(np.asarray(lats_array_era5_fixed_nh)[:,i])
        lats_month_era5_fixed_poleward=list(np.asarray(lats_array_era5_fixed_poleward_nh)[:,i])
        
    else:
        lats_month_era5_polar=lats_month_era5_polar+list(np.asarray(lats_array_era5_polar_nh)[:,i])
        lats_month_era5_fixed=lats_month_era5_fixed+list(np.asarray(lats_array_era5_fixed_nh)[:,i])
        lats_month_era5_fixed_poleward=lats_month_era5_fixed_poleward+list(np.asarray(lats_array_era5_fixed_poleward_nh)[:,i])

yrs=np.arange(1980,2024,1)
for i in range(len(yrs)):
    if i==0:
        lats_month_merra2_polar=list(np.asarray(lats_array_merra2_polar_nh)[:,i-1])
        lats_month_merra2_fixed=list(np.asarray(lats_array_merra2_fixed_nh)[:,i-1])
        lats_month_merra2_fixed_poleward=list(np.asarray(lats_array_merra2_fixed_poleward_nh)[:,i-1])
        
    else:
        lats_month_merra2_polar=lats_month_merra2_polar+list(np.asarray(lats_array_merra2_polar_nh)[:,i-1])
        lats_month_merra2_fixed=lats_month_merra2_fixed+list(np.asarray(lats_array_merra2_fixed_nh)[:,i-1])
        lats_month_merra2_fixed_poleward=lats_month_merra2_fixed_poleward+list(np.asarray(lats_array_merra2_fixed_poleward_nh)[:,i-1])

yrs=np.arange(1958,2024,1)
for i in range(len(yrs)):
    if i==0:
        lats_month_jra55_polar=list(np.asarray(lats_array_jra55_polar_nh)[:,i-1])
        lats_month_jra55_fixed=list(np.asarray(lats_array_jra55_fixed_nh)[:,i-1])
        lats_month_jra55_fixed_poleward=list(np.asarray(lats_array_jra55_fixed_poleward_nh)[:,i-1])
        
    else:
        lats_month_jra55_polar=lats_month_jra55_polar+list(np.asarray(lats_array_jra55_polar_nh)[:,i-1])
        lats_month_jra55_fixed=lats_month_jra55_fixed+list(np.asarray(lats_array_jra55_fixed_nh)[:,i-1])
        lats_month_jra55_fixed_poleward=lats_month_jra55_fixed_poleward+list(np.asarray(lats_array_jra55_fixed_poleward_nh)[:,i-1])

        
x_arr=np.arange(1979.5,2023.5,1./12.)


month_days=[31,28.25,31,30,31,30,31,31,30,31,30,31]


yrs=np.arange(1980,2024,1)
test=[x/1. for x in lats_list_merra2_fixed_nh]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_merra2_polar_nh]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='MERRA-2 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_merra2_fixed_poleward_nh]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='MERRA-2 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs1=np.arange(1940+40,2024,1)
test=[x/1. for x in lats_list_era5_fixed_nh[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_4='ERA5 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_era5_polar_nh[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_5='ERA5 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_era5_fixed_poleward_nh[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_6='ERA5 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs2=np.arange(1958+22,2024,1)
test=[x/1. for x in lats_list_jra55_fixed_nh[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_7='JRA-55 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_jra55_polar_nh[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_8='JRA-55 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_jra55_fixed_poleward_nh[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_9='JRA-55 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'


yrs1=np.arange(1940,2024,1)
yrs2=np.arange(1958,2024,1)

fig=plt.figure(figsize=(9,15))
ax=plt.subplot(1,1,1)

#MERRA-2: polar=salmon, fixed = yellow, fixed poleward = deepskyblue
#JRA-55: polar = r, fixed = gold, fixed poleward = royalblue
#ERA5: polar = darkred, fixed = orange, fixed poleward = mediumblue

#width=0.75
#ax.plot(x_arr,lats_month_merra2_polar,color='salmon',linewidth=width,alpha=0.5)
#ax.plot(x_arr,lats_month_merra2_fixed,color='yellow',linewidth=width,alpha=0.5)
#ax.plot(x_arr,lats_month_merra2_fixed_poleward,color='deepskyblue',linewidth=width,alpha=0.5)

#x_arr=np.arange(1957.5,2023.5,1./12.)
#ax.plot(x_arr,lats_month_jra55_polar,color='r',linewidth=width,alpha=0.5)
#ax.plot(x_arr,lats_month_jra55_fixed,color='gold',linewidth=width,alpha=0.5)
#ax.plot(x_arr,lats_month_jra55_fixed_poleward,color='royalblue',linewidth=width,alpha=0.5)

#x_arr=np.arange(1939.5,2023.5,1./12.)
#ax.plot(x_arr,lats_month_era5_polar,color='darkred',linewidth=width,alpha=0.5)
#ax.plot(x_arr,lats_month_era5_fixed,color='orange',linewidth=width,alpha=0.5)
#ax.plot(x_arr,lats_month_era5_fixed_poleward,color='mediumblue',linewidth=width,alpha=0.5)

ax.fill_between([1939,1979.5],37,49,color='gray',alpha=0.4)

width=8

ax.plot(yrs1,[x/1. for x in lats_list_era5_polar_nh],color='darkred',linewidth=width,label=label_5,zorder=10)
ax.plot(yrs2,[x/1. for x in lats_list_jra55_polar_nh],color='r',linewidth=width,label=label_8,zorder=13)
ax.plot(yrs,[x/1. for x in lats_list_merra2_polar_nh],color='salmon',linewidth=width,label=label_2,zorder=16)

ax.plot(yrs1,[x/1. for x in lats_list_era5_fixed_nh],color='orange',linewidth=width,label=label_4,zorder=11)
ax.plot(yrs2,[x/1. for x in lats_list_jra55_fixed_nh],color='gold',linewidth=width,label=label_7,zorder=14)
ax.plot(yrs,[x/1. for x in lats_list_merra2_fixed_nh],color='yellow',linewidth=width,label=label_1,zorder=17)

ax.plot(yrs1,[x/1. for x in lats_list_era5_fixed_poleward_nh],color='mediumblue',linewidth=width,label=label_6,zorder=12)
ax.plot(yrs2,[x/1. for x in lats_list_jra55_fixed_poleward_nh],color='royalblue',linewidth=width,label=label_9,zorder=15)
ax.plot(yrs,[x/1. for x in lats_list_merra2_fixed_poleward_nh],color='deepskyblue',linewidth=width,label=label_3,zorder=18)

width=2.5
yrs=np.arange(2000,2024,1)
test=lats_list_merra2_polar_nh[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_merra2_fixed_nh[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_merra2_fixed_poleward_nh[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=lats_list_era5_polar_nh[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_era5_fixed_nh[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_era5_fixed_poleward_nh[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=lats_list_jra55_polar_nh[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_jra55_fixed_nh[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_jra55_fixed_poleward_nh[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


y=np.arange(0,25,0.1)

ax.tick_params(labelsize=22)
ax.set_xlabel('Year',fontsize=24)
ax.set_ylabel('Degrees N/S',fontsize=24)
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{mean}$'+' '+r'$\bf{latitude:}$'+' '+r'$\bf{NH}$',fontsize=36,pad=5)
ax.grid(True)
plt.legend(loc='upper center',fontsize=16.4,ncol=1)
ax.set_xlim(1939.3,2023.6)
ax.set_ylim(38.01,47.5)
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.5)
plt.tight_layout()  
plt.show()

In [ ]:
fig.savefig(dir_data+'ar_obs_lines_4.png',dpi=300)

In [ ]:
yrs=np.arange(1940,2024,1)
for i in range(len(yrs)):
    if i==0:
        lats_month_era5_polar=list(np.asarray(lats_array_era5_polar_sh)[:,i])
        lats_month_era5_fixed=list(np.asarray(lats_array_era5_fixed_sh)[:,i])
        lats_month_era5_fixed_poleward=list(np.asarray(lats_array_era5_fixed_poleward_sh)[:,i])
        
    else:
        lats_month_era5_polar=lats_month_era5_polar+list(np.asarray(lats_array_era5_polar_sh)[:,i])
        lats_month_era5_fixed=lats_month_era5_fixed+list(np.asarray(lats_array_era5_fixed_sh)[:,i])
        lats_month_era5_fixed_poleward=lats_month_era5_fixed_poleward+list(np.asarray(lats_array_era5_fixed_poleward_sh)[:,i])

yrs=np.arange(1980,2024,1)
for i in range(len(yrs)):
    if i==0:
        lats_month_merra2_polar=list(np.asarray(lats_array_merra2_polar_sh)[:,i-1])
        lats_month_merra2_fixed=list(np.asarray(lats_array_merra2_fixed_sh)[:,i-1])
        lats_month_merra2_fixed_poleward=list(np.asarray(lats_array_merra2_fixed_poleward_sh)[:,i-1])
        
    else:
        lats_month_merra2_polar=lats_month_merra2_polar+list(np.asarray(lats_array_merra2_polar_sh)[:,i-1])
        lats_month_merra2_fixed=lats_month_merra2_fixed+list(np.asarray(lats_array_merra2_fixed_sh)[:,i-1])
        lats_month_merra2_fixed_poleward=lats_month_merra2_fixed_poleward+list(np.asarray(lats_array_merra2_fixed_poleward_sh)[:,i-1])

yrs=np.arange(1958,2024,1)
for i in range(len(yrs)):
    if i==0:
        lats_month_jra55_polar=list(np.asarray(lats_array_jra55_polar_sh)[:,i-1])
        lats_month_jra55_fixed=list(np.asarray(lats_array_jra55_fixed_sh)[:,i-1])
        lats_month_jra55_fixed_poleward=list(np.asarray(lats_array_jra55_fixed_poleward_sh)[:,i-1])
        
    else:
        lats_month_jra55_polar=lats_month_jra55_polar+list(np.asarray(lats_array_jra55_polar_sh)[:,i-1])
        lats_month_jra55_fixed=lats_month_jra55_fixed+list(np.asarray(lats_array_jra55_fixed_sh)[:,i-1])
        lats_month_jra55_fixed_poleward=lats_month_jra55_fixed_poleward+list(np.asarray(lats_array_jra55_fixed_poleward_sh)[:,i-1])

        
x_arr=np.arange(1979.5,2023.5,1./12.)


month_days=[31,28.25,31,30,31,30,31,31,30,31,30,31]


yrs=np.arange(1980,2024,1)
test=[x/1. for x in lats_list_merra2_fixed_sh]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_1='MERRA-2 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_merra2_polar_sh]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_2='MERRA-2 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_merra2_fixed_poleward_sh]
r=st.linregress(yrs,test)
sigs_list.append(r[3])
change=r[0]*len(yrs)
mean=float(sum(test))/float(len(test))
label_3='MERRA-2 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs1=np.arange(1940+40,2024,1)
test=[x/1. for x in lats_list_era5_fixed_sh[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_4='ERA5 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_era5_polar_sh[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_5='ERA5 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_era5_fixed_poleward_sh[40:]]
r=st.linregress(yrs1,test)
sigs_list.append(r[3])
change=r[0]*len(yrs1)
mean=float(sum(test))/float(len(test))
label_6='ERA5 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

yrs2=np.arange(1958+22,2024,1)
test=[x/1. for x in lats_list_jra55_fixed_sh[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_7='JRA-55 fixed (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_jra55_polar_sh[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_8='JRA-55 variable (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

test=[x/1. for x in lats_list_jra55_fixed_poleward_sh[22:]]
r=st.linregress(yrs2,test)
sigs_list.append(r[3])
change=r[0]*len(yrs2)
mean=float(sum(test))/float(len(test))
label_9='JRA-55 fixed poleward (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'


yrs1=np.arange(1940,2024,1)
yrs2=np.arange(1958,2024,1)

fig=plt.figure(figsize=(9,15))
ax=plt.subplot(1,1,1)

#MERRA-2: polar=salmon, fixed = yellow, fixed poleward = deepskyblue
#JRA-55: polar = r, fixed = gold, fixed poleward = royalblue
#ERA5: polar = darkred, fixed = orange, fixed poleward = mediumblue

#width=0.75
#ax.plot(x_arr,lats_month_merra2_polar,color='salmon',linewidth=width,alpha=0.5)
#ax.plot(x_arr,lats_month_merra2_fixed,color='yellow',linewidth=width,alpha=0.5)
#ax.plot(x_arr,lats_month_merra2_fixed_poleward,color='deepskyblue',linewidth=width,alpha=0.5)

#x_arr=np.arange(1957.5,2023.5,1./12.)
#ax.plot(x_arr,lats_month_jra55_polar,color='r',linewidth=width,alpha=0.5)
#ax.plot(x_arr,lats_month_jra55_fixed,color='gold',linewidth=width,alpha=0.5)
#ax.plot(x_arr,lats_month_jra55_fixed_poleward,color='royalblue',linewidth=width,alpha=0.5)

#x_arr=np.arange(1939.5,2023.5,1./12.)
#ax.plot(x_arr,lats_month_era5_polar,color='darkred',linewidth=width,alpha=0.5)
#ax.plot(x_arr,lats_month_era5_fixed,color='orange',linewidth=width,alpha=0.5)
#ax.plot(x_arr,lats_month_era5_fixed_poleward,color='mediumblue',linewidth=width,alpha=0.5)

ax.fill_between([1939,1979.5],37,49,color='gray',alpha=0.4)

width=8

ax.plot(yrs1,[x/1. for x in lats_list_era5_polar_sh],color='darkred',linewidth=width,label=label_5,zorder=10)
ax.plot(yrs2,[x/1. for x in lats_list_jra55_polar_sh],color='r',linewidth=width,label=label_8,zorder=13)
ax.plot(yrs,[x/1. for x in lats_list_merra2_polar_sh],color='salmon',linewidth=width,label=label_2,zorder=16)

ax.plot(yrs1,[x/1. for x in lats_list_era5_fixed_sh],color='orange',linewidth=width,label=label_4,zorder=11)
ax.plot(yrs2,[x/1. for x in lats_list_jra55_fixed_sh],color='gold',linewidth=width,label=label_7,zorder=14)
ax.plot(yrs,[x/1. for x in lats_list_merra2_fixed_sh],color='yellow',linewidth=width,label=label_1,zorder=17)

ax.plot(yrs1,[x/1. for x in lats_list_era5_fixed_poleward_sh],color='mediumblue',linewidth=width,label=label_6,zorder=12)
ax.plot(yrs2,[x/1. for x in lats_list_jra55_fixed_poleward_sh],color='royalblue',linewidth=width,label=label_9,zorder=15)
ax.plot(yrs,[x/1. for x in lats_list_merra2_fixed_poleward_sh],color='deepskyblue',linewidth=width,label=label_3,zorder=18)

width=2.5
yrs=np.arange(2000,2024,1)
test=lats_list_merra2_polar_sh[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_merra2_fixed_sh[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_merra2_fixed_poleward_sh[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=lats_list_era5_polar_sh[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_era5_fixed_sh[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_era5_fixed_poleward_sh[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


yrs=np.arange(2000,2024,1)
test=lats_list_jra55_polar_sh[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_jra55_fixed_sh[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)

test=lats_list_jra55_fixed_poleward_sh[-24:]
r=st.linregress(yrs,test)
if r[3]<=0.05:
    style='--'
else:
    style=':'
ax.plot(yrs,[r[0]*x+r[1] for x in yrs],color='k',linewidth=width,linestyle=style,zorder=20)


y=np.arange(0,25,0.1)

ax.tick_params(labelsize=22)
ax.set_xlabel('Year',fontsize=24)
ax.set_ylabel('Degrees S',fontsize=24)
ax.set_title(r'$\bf{AR}$'+' '+r'$\bf{mean}$'+' '+r'$\bf{latitude:}$'+' '+r'$\bf{SH}$',fontsize=36,pad=5)
ax.grid(True)
plt.legend(loc='upper center',fontsize=16.4,ncol=1)
ax.set_xlim(1939.3,2023.6)
ax.set_ylim(38.01,47.5)
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.5)
plt.tight_layout()  
plt.show()

In [ ]:
fig.savefig(dir_data+'ar_obs_lines_5.png',dpi=300)

In [ ]:
print(sigs_list)

In [ ]:
print(len(sigs_list))